In [1]:
import pandas as pd
import numpy as np
import glob
import os
import plotly.express as px
import plotly.graph_objects as go
import pickle
import base64

In [2]:
#ForKK
DataPath = "./XC-CleanedData/**/*.csv"
FilteredDataPath = "./GeneratedData/"
FilteredDataPathPlayback = "./GeneratedData/playback/"
AnimPath = "./GeneratedAnimations/"

In [3]:
ListOfScenarios=["CP1","CP2","CP3","CP5","CP6","CP7","CP8"]
MaxDistance=35
window_size = 5  # This can be changed depending on how much smoothing you want
kernel = np.ones(window_size) / window_size
OutputColumns = ["ApproachRate", "ApproachRateOther", "Rel_Pos_Magnitude", 
                 "ScenarioTime", 
                 "AccelA","AccelB","SteerA","SteerB",
                 "A_Head_Center_Distance","B_Head_Center_Distance",
                 "Filtered_A_Head_Velocity_Total","Filtered_B_Head_Velocity_Total",
                 "RelativeRotation"]
column_types = {'ScenarioTime': 'float64',
                'AccelA': 'float64',
                'AccelB': 'float64',
                'SteerA': 'float64',
                'SteerB': 'float64',
                'HeadPosXA': 'float64',
                'HeadPosYA': 'float64',
                'HeadPosZA': 'float64',
                'HeadPosXB': 'float64',
                'HeadPosYB': 'float64',
                'HeadPosZB': 'float64',
                'HeadrotYA': 'float64',
                'HeadrotYB': 'float64',
                }

required_columns = set(column_types.keys())

In [4]:
def plot_and_save(Scenario, run, innerArea):
    # Create a DataFrame for plotting
    plot_df = pd.DataFrame({
        'Time': innerArea['ScenarioTime'],
        'A_HeadPosX': innerArea['HeadPosXA'],
        'A_HeadPosZ': innerArea['HeadPosZA'],
        'B_HeadPosX': innerArea['HeadPosXB'],
        'B_HeadPosZ': innerArea['HeadPosZB']
    })

    # Melt the DataFrame for easier plotting with plotly.express
    plot_df = plot_df.melt(id_vars=['Time'], value_vars=['A_HeadPosX', 'A_HeadPosZ', 'B_HeadPosX', 'B_HeadPosZ'], 
                        var_name='variable', value_name='value')

    # Extract entity and coordinate information from 'variable'
    plot_df[['Entity', 'Coordinate']] = plot_df['variable'].str.extract(r'([AB])_(HeadPos[XYZ])')

    # Separate DataFrames for X and Z coordinates
    df_x = plot_df[plot_df['Coordinate'] == 'HeadPosX']
    df_z = plot_df[plot_df['Coordinate'] == 'HeadPosZ']

    # Merge X and Z coordinates
    plot_df = df_x.merge(df_z, on=['Time', 'Entity'], suffixes=('_X', '_Z'))

    # Create animated scatter plot with plotly.express
    fig = px.scatter(plot_df, x='value_X', y='value_Z', animation_frame='Time', animation_group='Entity', color='Entity',
                    labels={'value_X': 'X-axis', 'value_Z': 'Z-axis'}, range_x=[-MaxDistance, MaxDistance], range_y=[-MaxDistance, MaxDistance])

    fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = plot_df['Time'].unique()[1]*1000
    # Add the circle representing the maximum distance using graph_objects
    fig.add_shape(type="circle",
                x0=-MaxDistance, y0=-MaxDistance,
                x1=MaxDistance, y1=MaxDistance,
                line=dict(color="blue"))

    # Update layout to maintain aspect ratio and add title
    fig.update_layout(
        title=f"Scenario: {Scenario}, Run: {run}",
        yaxis_scaleanchor="x",
        yaxis_scaleratio=1
    )
    img1 = base64.b64encode(open('scene.jpeg', 'rb').read())
    fig.update_layout(
            images= [dict(
                source='data:image/png;base64,{}'.format(img1.decode()),
                xref="x", yref="y",
                x=-100, y=120,
                sizex=190, sizey=240,
                xanchor="left",
                yanchor="top",
                sizing="fill",
                opacity=0.8,
                layer="below")]
                )

    # Save the plot as an HTML file
    if not os.path.exists(AnimPath):
        os.makedirs(AnimPath)
    html_path = os.path.join(AnimPath, f"{Scenario}_{run}.html")
    fig.write_html(html_path)
    print(f"Animation saved at: {html_path}")


In [5]:
pd.options.mode.chained_assignment = None
CSVFiles = glob.glob(DataPath, recursive=True)
FinalFileList = []

In [6]:
for val in CSVFiles:
    Scenario = os.path.basename(val).split('-')[1].split('_')[0]
    run = os.path.basename(val).split('-')[2].split('_')[0]
    if Scenario in ListOfScenarios:
        try:
            df = pd.read_csv(val, sep=';', dtype=column_types)
        except Exception as e:
            print(f"An error occurred while reading {Scenario} at {run}: {e}")
            continue
        is_columns = set(df.columns)
        if not required_columns.issubset(is_columns):
            missing_columns = required_columns - is_columns
            print(f"DataFrame is missing columns: {missing_columns} {Scenario}{run}")
            continue
        # Head-Center Distance
        df['A_Head_Center_Distance'] = np.sqrt(df['HeadPosXA']**2 + df['HeadPosYA']**2 + df['HeadPosZA']**2)
        df['B_Head_Center_Distance'] = np.sqrt(df['HeadPosXB']**2 + df['HeadPosYB']**2 + df['HeadPosZB']**2)
        # Define dataframe inner area to be within a radius of MaxDistance = 35
        innerArea = df.loc[(df['A_Head_Center_Distance'] < MaxDistance)
                            & (df['B_Head_Center_Distance'] < MaxDistance)].copy().reset_index()
        if innerArea.shape[0] == 0:
            print("Skipping: ", Scenario, run)
            continue
        # Timeframe reset
        innerArea['DeltaTime'] = innerArea['ScenarioTime'].diff()
        innerArea['ScenarioTime']= innerArea['ScenarioTime'] - innerArea['ScenarioTime'].iloc[0]
        # todo: turn head_delta into vector
        # Delta positions along x,y,z axis
        innerArea['A_Head_DeltaX'] = innerArea['HeadPosXA'].diff()
        innerArea['A_Head_DeltaZ'] = innerArea['HeadPosZA'].diff()
        innerArea['A_Head_DeltaY'] = innerArea['HeadPosYA'].diff()
        innerArea['B_Head_DeltaX'] = innerArea['HeadPosXB'].diff()
        innerArea['B_Head_DeltaY'] = innerArea['HeadPosYB'].diff()
        innerArea['B_Head_DeltaZ'] = innerArea['HeadPosZB'].diff()
        # Velocity along x,y,z axis
        innerArea['A_Head_VelocityX'] = innerArea['A_Head_DeltaX'] / innerArea['DeltaTime']
        innerArea['A_Head_VelocityY'] = innerArea['A_Head_DeltaY'] / innerArea['DeltaTime']
        innerArea['A_Head_VelocityZ'] = innerArea['A_Head_DeltaZ'] / innerArea['DeltaTime']
        innerArea['B_Head_VelocityX'] = innerArea['B_Head_DeltaX'] / innerArea['DeltaTime']
        innerArea['B_Head_VelocityY'] = innerArea['B_Head_DeltaY'] / innerArea['DeltaTime']
        innerArea['B_Head_VelocityZ'] = innerArea['B_Head_DeltaZ'] / innerArea['DeltaTime']
        # Set start velocity at 0
        innerArea['A_Head_VelocityX'].iloc[0] =0#innerArea['A_Head_VelocityX'].iloc[1]
        innerArea['A_Head_VelocityY'].iloc[0] =0#innerArea['A_Head_VelocityY'].iloc[1]
        innerArea['A_Head_VelocityZ'].iloc[0] =0#innerArea['A_Head_VelocityZ'].iloc[1]
        innerArea['B_Head_VelocityX'].iloc[0] =0#innerArea['B_Head_VelocityX'].iloc[1]
        innerArea['B_Head_VelocityY'].iloc[0] =0#innerArea['B_Head_VelocityY'].iloc[1]
        innerArea['B_Head_VelocityZ'].iloc[0] =0#innerArea['B_Head_VelocityZ'].iloc[1]
        # Smooth velocity and get total velocity
        innerArea['Filtered_A_Head_VelocityX'] = np.convolve(innerArea['A_Head_VelocityX'], kernel, 'same')
        innerArea['Filtered_A_Head_VelocityY'] = np.convolve(innerArea['A_Head_VelocityY'], kernel, 'same')
        innerArea['Filtered_A_Head_VelocityZ'] = np.convolve(innerArea['A_Head_VelocityZ'], kernel, 'same')
        innerArea['Filtered_A_Head_Velocity_Total'] = np.sqrt(innerArea['Filtered_A_Head_VelocityX']**2 
                                                            + innerArea['Filtered_A_Head_VelocityY']**2 
                                                            + innerArea['Filtered_A_Head_VelocityZ']**2)
        innerArea['Filtered_B_Head_VelocityX'] = np.convolve(innerArea['B_Head_VelocityX'], kernel, 'same')
        innerArea['Filtered_B_Head_VelocityY'] = np.convolve(innerArea['B_Head_VelocityY'], kernel, 'same')
        innerArea['Filtered_B_Head_VelocityZ'] = np.convolve(innerArea['B_Head_VelocityZ'], kernel, 'same')
        innerArea['Filtered_B_Head_Velocity_Total'] = np.sqrt(innerArea['Filtered_B_Head_VelocityX']**2 
                                                            + innerArea['Filtered_B_Head_VelocityY']**2 
                                                            + innerArea['Filtered_B_Head_VelocityZ']**2)
        # Relative velocity along x,y,z axis
        innerArea['RelVelocity_X'] = innerArea['Filtered_A_Head_VelocityX'] - innerArea['Filtered_B_Head_VelocityX']
        innerArea['RelVelocity_Y'] = innerArea['Filtered_A_Head_VelocityY'] - innerArea['Filtered_B_Head_VelocityY']
        innerArea['RelVelocity_Z'] = innerArea['Filtered_A_Head_VelocityZ'] - innerArea['Filtered_B_Head_VelocityZ']
        # Relative total velocity
        innerArea['RelVelocity'] = np.sqrt(innerArea['RelVelocity_X']**2 
                                            + innerArea['RelVelocity_Y']**2 
                                            + innerArea['RelVelocity_Z']**2)
        # Relative distance along x,y,z axis
        innerArea['Rel_Distance_X'] = innerArea['HeadPosXA'] - innerArea['HeadPosXB']
        innerArea['Rel_Distance_Y'] = innerArea['HeadPosYA'] - innerArea['HeadPosYB']
        innerArea['Rel_Distance_Z'] = innerArea['HeadPosZA'] - innerArea['HeadPosZB']
        # Relative position distance
        innerArea['Rel_Pos_Magnitude'] = np.sqrt(innerArea['Rel_Distance_X']**2 
                                                + innerArea['Rel_Distance_Y']**2 
                                                + innerArea['Rel_Distance_Z']**2)
        # Measures the component of the relative velocity that is in the direction of the relative distance
        innerArea['Dot_Product'] = (innerArea['Rel_Distance_X'] * innerArea['RelVelocity_X'] 
                                    + innerArea['Rel_Distance_Y'] * innerArea['RelVelocity_Y'] 
                                    + innerArea['Rel_Distance_Z'] * innerArea['RelVelocity_Z'])
        # Normalizes this measure by the distance, 
        # providing a rate at which the two entities are closing in on each other
        innerArea['ApproachRate'] = innerArea['Dot_Product'] / innerArea['Rel_Pos_Magnitude']
        # Measures the component of the smoothed relative velocity that is in the direction of the relative distance
        innerArea['Dot_Product_Other'] = (innerArea['Rel_Distance_X'] * innerArea['Filtered_B_Head_VelocityX'] 
                                        + innerArea['Rel_Distance_Y'] * innerArea['Filtered_B_Head_VelocityY'] 
                                        + innerArea['Rel_Distance_Z'] * innerArea['Filtered_B_Head_VelocityZ'])
        # providing a rate at which the two entities are closing in on each other after smoothing
        innerArea['ApproachRateOther'] = innerArea['Dot_Product_Other'] / innerArea['Rel_Pos_Magnitude']
        # ToDo: Add intended Driving direction as a variable (left=>-1 right=>1)

        # ToDo: Generate a centerLine(Either through average or by hand)
        # ToDo: compute offset from center line
        # ToDo: add Car B's indicator into -1=>1
        # ToDo: Contemplate other line depended varialbes
        innerArea['RelativeRotation'] = innerArea['HeadrotYA']- innerArea['HeadrotYB']
        plot_and_save(Scenario, run, innerArea)
        
        # path = os.path.join(FilteredDataPath,Scenario+'_'+run+'.feather')
        # innerArea[OutputColumns].to_feather(path)
        # pathPlayback = os.path.join(FilteredDataPathPlayback,Scenario+'_'+run+'.csv')
        # innerArea.to_csv(pathPlayback)
        # FinalFileList.append(path)

Skipping:  CP5 NYC1
Skipping:  CP2 NYC1
Skipping:  CP1 NYC1
Skipping:  CP1 NYC25
Skipping:  CP1 ISR13
Skipping:  CP1 NYC9
Skipping:  CP5 ISR39
Skipping:  CP5 ISR38
Skipping:  CP2 ISR38
Skipping:  CP3 NYC2
Skipping:  CP2 NYC10
Skipping:  CP6 ISR11
DataFrame is missing columns: {'AccelA', 'HeadPosZA', 'HeadrotYB', 'HeadPosXA', 'HeadPosYA', 'ScenarioTime', 'SteerB', 'HeadPosXB', 'HeadrotYA', 'SteerA', 'AccelB', 'HeadPosZB', 'HeadPosYB'} CP1NYC42
Skipping:  CP3 NYC11
Skipping:  CP7 NYC16
Skipping:  CP7 NYC29
Skipping:  CP5 NYC3
Skipping:  CP3 NYC4
Skipping:  CP5 ITH3
Skipping:  CP1 NYC34
Skipping:  CP1 NYC34
An error occurred while reading CP1 at NYC34: could not convert string to float: ''
An error occurred while reading CP7 at NYC33: could not convert string to float: ''
An error occurred while reading CP2 at NYC33: could not convert string to float: ''
Skipping:  CP6 NYC14B
Skipping:  CP6 ISR34
Skipping:  CP5 ISR03


In [ ]:
plot_and_save(Scenario, run, innerArea)